In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [22]:
NCAA_compact = pd.read_csv('./DataFiles/NCAATourneyCompactResults.csv')
NCAA_detailed = pd.read_csv('./DataFiles/NCAATourneyDetailedResults.csv')
reg_compact = pd.read_csv('./DataFiles/RegularSeasonCompactResults.csv')
reg_detailed = pd.read_csv('./DataFiles/RegularSeasonDetailedResults.csv')
secondary_compact = pd.read_csv('./DataFiles/SecondaryTourneyCompactResults.csv')
secondary_detailed = pd.read_csv('./DataFiles/Sec')

In [23]:
'''
Some notes/ideas if you’re doing NCAA:

1. The project requires a lot of merging/concatenating of DataFrames. Checkout the lecture from week 1 on this if you need a refresher: \
https://www.youtube.com/watch?v=_9fpBhqKdXw&index=4&t=275s&list=PLYGvOQTnahYLD-A-NIUpvK-8E2B0GXBE8

2. You can use season averages from `RegularSeasonCompactResults.csv` and `RegularSeasonDetailedResults.csv` \
as features for `NCAATourneyCompactResults.csv` and `NCAATourneyDetailedResults.csv` respectively\
(hint: `.groupby()`, `.mean()` and `.merge()`)

3. The data from `NCAATourneySeeds.csv` can be merged as features for the tournament games

'''

'\nSome notes/ideas if you’re doing NCAA:\n\n1. The project requires a lot of merging/concatenating of DataFrames. Checkout the lecture from week 1 on this if you need a refresher: https://www.youtube.com/watch?v=_9fpBhqKdXw&index=4&t=275s&list=PLYGvOQTnahYLD-A-NIUpvK-8E2B0GXBE8\n\n2. You can use season averages from `RegularSeasonCompactResults.csv` and `RegularSeasonDetailedResults.csv` as features for `NCAATourneyCompactResults.csv` and `NCAATourneyDetailedResults.csv` respectively(hint: `.groupby()`, `.mean()` and `.merge()`)\n\n3. The data from `NCAATourneySeeds.csv` can be merged as features for the tournament games\n\n'

In [24]:
WLoc = {'A': 1, 'H': 2, 'N': 3}
Secondary_Tourney = {'NIT': 1, 'CBI': 2, 'CIT': 3, 'V16': 4, 'Regular': 5 ,'NCAA': 6}

## Generating massive DataFrame.


In [25]:
games = pd.concat((NCAA_compact,reg_compact), axis = 0, ignore_index=True)

games = pd.concat((games, secondary_compact), axis=0, ignore_index=True)



In [26]:
games.head()

,DayNum,LScore,LTeamID,NumOT,Season,SecondaryTourney,WLoc,WScore,WTeamID
0,136,54,1234,0,1985,NaN,N,63,1116
1,136,58,1345,0,1985,NaN,N,59,1120
2,136,43,1250,0,1985,NaN,N,68,1207
3,136,55,1425,0,1985,NaN,N,58,1229
4,136,38,1325,0,1985,NaN,N,49,1242


In [27]:
results = []

for row in games.to_dict('records'):
    for perspective in ['W', 'L']:
        game = {}
        game['Won'] = int(perspective == 'W')
        if perspective == 'W':
            team_letter = 'W'
            opp_letter = 'L'
        else:
            team_letter = 'L'
            opp_letter = 'W'
        winner_location = row["WLoc"]
        if winner_location == ' A':
            loser_location = 'H'
        elif winner_location == 'H':
            loser_location = 'A'
        else:
            loser_location = 'N'
        row['LLoc'] = loser_location
        team_stats = [k for k,v in row.items() if k[0] == team_letter]
        opp_stats = [k for k,v in row.items() if k[0] == opp_letter]
    #     print(team_stats)
    #     print(opp_stats)

        for stat in team_stats:
            game_stat = 'Team' + stat[1:]
            game[game_stat] = row[stat]

        for stat in opp_stats:
            opp_stat = 'Opp' + stat[1:]
            game[opp_stat] = row[stat]

        for general_stat in ['DayNum', 'NumOT', 'Season']:
            game[general_stat] = row[general_stat]

        results.append(game)
    
results

[{'DayNum': 136,
  'NumOT': 0,
  'OppLoc': 'N',
  'OppScore': 54,
  'OppTeamID': 1234,
  'Season': 1985,
  'TeamLoc': 'N',
  'TeamScore': 63,
  'TeamTeamID': 1116,
  'Won': 1},
 {'DayNum': 136,
  'NumOT': 0,
  'OppLoc': 'N',
  'OppScore': 63,
  'OppTeamID': 1116,
  'Season': 1985,
  'TeamLoc': 'N',
  'TeamScore': 54,
  'TeamTeamID': 1234,
  'Won': 0},
 {'DayNum': 136,
  'NumOT': 0,
  'OppLoc': 'N',
  'OppScore': 58,
  'OppTeamID': 1345,
  'Season': 1985,
  'TeamLoc': 'N',
  'TeamScore': 59,
  'TeamTeamID': 1120,
  'Won': 1},
 {'DayNum': 136,
  'NumOT': 0,
  'OppLoc': 'N',
  'OppScore': 59,
  'OppTeamID': 1120,
  'Season': 1985,
  'TeamLoc': 'N',
  'TeamScore': 58,
  'TeamTeamID': 1345,
  'Won': 0},
 {'DayNum': 136,
  'NumOT': 0,
  'OppLoc': 'N',
  'OppScore': 43,
  'OppTeamID': 1250,
  'Season': 1985,
  'TeamLoc': 'N',
  'TeamScore': 68,
  'TeamTeamID': 1207,
  'Won': 1},
 {'DayNum': 136,
  'NumOT': 0,
  'OppLoc': 'N',
  'OppScore': 68,
  'OppTeamID': 1207,
  'Season': 1985,
  'TeamLoc

In [28]:
games_df = pd.DataFrame(results)

In [30]:
games_df.head()
games_df.shape

(308570, 10)

## An even bigger DataFrame.

In [31]:
big_df = pd.concat((NCAA_detailed, reg_detailed), axis = 0, ignore_index=True)

big_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [32]:
big_df.shape

(77617, 34)

In [33]:
results = []

for row in big_df.to_dict('records'):
    for perspective in ['W', 'L']:
        game = {}
        game['Won'] = int(perspective == 'W')
        if perspective == 'W':
            team_letter = 'W'
            opp_letter = 'L'
        else:
            team_letter = 'L'
            opp_letter = 'W'
        winner_location = row["WLoc"]
        if winner_location == ' A':
            loser_location = 'H'
        elif winner_location == 'H':
            loser_location = 'A'
        else:
            loser_location = 'N'
        row['LLoc'] = loser_location
        team_stats = [k for k,v in row.items() if k[0] == team_letter]
        opp_stats = [k for k,v in row.items() if k[0] == opp_letter]
    #     print(team_stats)
    #     print(opp_stats)

        for stat in team_stats:
            game_stat = 'Team' + stat[1:]
            game[game_stat] = row[stat]

        for stat in opp_stats:
            opp_stat = 'Opp' + stat[1:]
            game[opp_stat] = row[stat]

        for general_stat in ['DayNum', 'NumOT', 'Season']:
            game[general_stat] = row[general_stat]

        results.append(game)
    
results

[{'DayNum': 134,
  'NumOT': 1,
  'OppAst': 16,
  'OppBlk': 0,
  'OppDR': 28,
  'OppFGA': 67,
  'OppFGA3': 31,
  'OppFGM': 29,
  'OppFGM3': 12,
  'OppFTA': 31,
  'OppFTM': 14,
  'OppLoc': 'N',
  'OppOR': 17,
  'OppPF': 22,
  'OppScore': 84,
  'OppStl': 5,
  'OppTO': 15,
  'OppTeamID': 1411,
  'Season': 2003,
  'TeamAst': 17,
  'TeamBlk': 3,
  'TeamDR': 30,
  'TeamFGA': 69,
  'TeamFGA3': 29,
  'TeamFGM': 32,
  'TeamFGM3': 11,
  'TeamFTA': 26,
  'TeamFTM': 17,
  'TeamLoc': 'N',
  'TeamOR': 14,
  'TeamPF': 22,
  'TeamScore': 92,
  'TeamStl': 5,
  'TeamTO': 12,
  'TeamTeamID': 1421,
  'Won': 1},
 {'DayNum': 134,
  'NumOT': 1,
  'OppAst': 17,
  'OppBlk': 3,
  'OppDR': 30,
  'OppFGA': 69,
  'OppFGA3': 29,
  'OppFGM': 32,
  'OppFGM3': 11,
  'OppFTA': 26,
  'OppFTM': 17,
  'OppLoc': 'N',
  'OppOR': 14,
  'OppPF': 22,
  'OppScore': 92,
  'OppStl': 5,
  'OppTO': 12,
  'OppTeamID': 1421,
  'Season': 2003,
  'TeamAst': 16,
  'TeamBlk': 0,
  'TeamDR': 28,
  'TeamFGA': 67,
  'TeamFGA3': 31,
  'TeamFG

In [34]:
games_df_2 = pd.DataFrame(results)

In [35]:
games_df_2.shape

(155234, 36)

In [40]:
seeds = pd.read_csv('./DataFiles/NCAATourneySeeds.csv')

seeds = {'_'.join(map(str,[int(k1), k2])): int(v[1:3]) for k1, v, k2 in seeds.values }